<a href="https://colab.research.google.com/github/harsh-hy/computer-vison/blob/main/comp_vis_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pylab import gray, imshow, show

In [ ]:
imgl01=cv2.imread('left01.jpg' ,cv2.IMREAD_GRAYSCALE)
imgr01=cv2.imread('right01.jpg',cv2.IMREAD_GRAYSCALE)
imgl02=cv2.imread('left02.jpg' ,cv2.IMREAD_GRAYSCALE)
imgr02=cv2.imread('right02.jpg',cv2.IMREAD_GRAYSCALE)

In [ ]:
imshow(imgl01)
show()


In [ ]:
imshow(imgl02)
show()

In [ ]:

imshow(imgr01)
show()

In [ ]:

imshow(imgr02)
show()

In [ ]:
stero=cv2.StereoBM_create(numDisparities=64, blockSize=15)
disparity=stero.compute(imgl01,imgr01)
disparity_norm=cv2.normalize(src=disparity,dst=disparity,beta=0,alpha=255,norm_type=cv2.NORM_MINMAX)
disparity_norm=np.uint8(disparity_norm)
plt.figure(figsize=(10,10))
plt.imshow(disparity_norm,cmap='plasma')
plt.colorbar(label="Disparity")
plt.title("Disparity Map")
plt.show()

In [ ]:
orb=cv2.ORB_create(nfeatures=1000)
kp1,des1=orb.detectAndCompute(imgl01,None)
kp2,des2=orb.detectAndCompute(imgr01,None)

In [ ]:
bf=cv2.BFMatcher(cv2.NORM_HAMMING,crossCheck=True)
matches=bf.match(des1,des2)
matches=sorted(matches,key=lambda x:x.distance)

In [ ]:
pts1=np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1,1,2)
pts2=np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1,1,2)

In [ ]:
f, mask=cv2.findFundamentalMat(pts1,pts2,cv2.FM_RANSAC)

In [ ]:
pts1=pts1[mask.ravel()==1]
pts2=pts2[mask.ravel()==1]

In [ ]:
print("Computed Fundamental Matrix")
print(f)

In [ ]:
def draw_epipolar_lines(imgl01,imgr01,f,pts1,pts2):
    lines1=cv2.computeCorrespondEpilines(pts2.reshape(-1,1,2),2,f)
    lines1=lines1.reshape(-1,3)

    img1_color=cv2.cvtColor(imgl01,cv2.COLOR_GRAY2BGR)
    img2_color=cv2.cvtColor(imgr01,cv2.COLOR_GRAY2BGR)

    for r,pt1,pt2 in zip(lines1,pts1,pts2):
      color=tuple(np.random.randint(0,255,3).tolist())
      x0,y0=map(int, [0,-r[2]/r[1]])
      x1,y1=map(int, [imgl01.shape[1],-(r[2]+r[0]*imgl01.shape[1])/r[1]])
      img1_color=cv2.line(img1_color,(x0,y0),(x1,y1),color,1)
      img1_color = cv2.circle(img1_color, tuple(pt1[0].astype(int)), 5, color, -1)
    plt.figure(figsize=(10,10))
    plt.subplot(121),plt.imshow(img1_color)
    plt.subplot(122),plt.imshow(img2_color)
    plt.show()
draw_epipolar_lines(imgl01,imgr01,f,pts1,pts2)
